In [ ]:
%matplotlib inline
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import flopy
import pyemu
import prep_deps
import redis
import matplotlib as mpl
plt.rcParams['font.size']=12

In [ ]:
org_d = os.path.join("..","base_model_files")
org_nam = "freyberg.nam"

In [ ]:
m_org = flopy.modflow.Modflow.load(org_nam,model_ws=org_d,check=False)

In [ ]:
tr_d = "temp_history"
tr_nam = "freyberg"

In [ ]:
m_tr = flopy.modflow.Modflow(tr_nam,model_ws=tr_d,version="mfnwt")

In [ ]:
tr_nper = 24 #num transient stress periods
freq = "m"
steady = [True]
for _ in range(tr_nper):
    steady.append(False)
start_datetime = "12-31-2015"
end = pd.date_range(start=start_datetime,periods=tr_nper+1,freq=freq)
delt = end[1:] - end[:-1]
perlen = list(delt.days.values)
perlen.insert(0,1)
print(end)
print(perlen)
model_start_datetime = "12-31-2015"
assert len(perlen) == tr_nper + 1,len(perlen)

In [ ]:
sp_end = pd.to_datetime(model_start_datetime) + pd.to_timedelta(np.cumsum(np.array(perlen)),unit='d')
sp_end

In [ ]:
_ = flopy.modflow.ModflowDis(m_tr,nper=tr_nper+1,nlay=m_org.nlay,nrow=m_org.nrow,ncol=m_org.ncol,delr=m_org.dis.delr.array,
                        delc=m_org.dis.delc.array,top=m_org.dis.top.array,botm=m_org.dis.botm.array,steady=steady,
                        perlen=perlen)
m_tr.dis.start_datetime = model_start_datetime

In [ ]:
_ = flopy.modflow.ModflowBas(m_tr,ibound=m_org.bas6.ibound.array,strt=m_org.bas6.strt.array,hnoflo=m_org.bas6.hnoflo)

In [ ]:
_ = flopy.modflow.ModflowUpw(m_tr,ipakcb=50,laytyp=[1,0,0],hk=m_org.upw.hk.array,
                             vka=m_org.upw.vka.array,ss=m_org.upw.ss.array,sy=m_org.upw.sy.array)

In [ ]:
_ = flopy.modflow.ModflowNwt(m_tr,headtol=0.01,fluxtol=1.0)
_ = flopy.modflow.ModflowOc(m_tr,stress_period_data={(kper,0):["save head","save budget"] for kper in range(m_tr.nper)})

In [ ]:
angles = np.linspace(-np.pi, np.pi, tr_nper)
season_mults = 1.0 + 0.65*np.sin(1 + angles*2)
wel_season_mults = np.roll(season_mults,int(tr_nper / 4))
plt.plot(np.arange(tr_nper),season_mults)
plt.plot(np.arange(tr_nper),wel_season_mults)


In [ ]:
org_wel_data = m_org.wel.stress_period_data[0]
org_rch = m_org.rch.rech[0].array
wel_data = {0:org_wel_data}
rech = {0:org_rch}
for kper in range(1,m_tr.nper):
    kper_wel_data = org_wel_data.copy()
    kper_wel_data["flux"] *= wel_season_mults[kper-1]
    wel_data[kper] = kper_wel_data
    rech[kper] = org_rch * season_mults[kper-1]

In [ ]:
_ = flopy.modflow.ModflowWel(m_tr,stress_period_data=wel_data,ipakcb=50)

In [ ]:
_ = flopy.modflow.ModflowRch(m_tr,rech=rech,ipakcb=50)

In [ ]:
_ = flopy.modflow.ModflowDrn(m_tr,stress_period_data=m_org.drn.stress_period_data,ipakcb=50)

In [ ]:
m_org.sfr.reach_data

In [ ]:
rdata = pd.DataFrame.from_records(m_org.sfr.reach_data)
sdata = pd.DataFrame.from_records(m_org.sfr.segment_data[0])
#print(rdata.reachID)

rdata = rdata.reindex(np.arange(m_tr.nrow))
#print(rdata.strthick)
#return
# rdata.loc[:,'k'] = 0
# rdata.loc[:,'j'] = 16
# rdata.loc[:,'rchlen'] = mfr.dis.delc.array
# rdata.loc[:,'i'] = np.arange(mfr.nrow)
# rdata.loc[:,"iseg"] = rdata.i + 1
# rdata.loc[:,"ireach"] = 1
# rdata.loc[:,"reachID"] = rdata.index.values
# rdata.loc[:,"outreach"] = rdata.reachID + 1
# rdata.loc[mfr.nrow-1,"outreach"] = 0
# rdata.loc[:,"node"] = rdata.index.values
# for col in ["strthick","thts","thti","eps","uhc","strhc1"]:
#     rdata.loc[:,col] = rdata.loc[0,col]



upstrm = 34
dwstrm = 33.5
total_length = m_tr.dis.delc.array.max() * m_tr.nrow
slope = (upstrm - dwstrm) / total_length
# print(rdata.dtype,slope)
strtop = np.linspace(upstrm, dwstrm, m_tr.nrow)
# print(strtop)
rdata.loc[:,"strtop"] = strtop
rdata.loc[:,"slope"] = slope

#print(sdata.nseg)
sdata = sdata.reindex(np.arange(m_tr.nrow))
for column in sdata.columns:
    sdata.loc[:,column] = sdata.loc[0,column]
sdata.loc[:,"nseg"] = np.arange(m_tr.nrow) + 1
sdata.loc[1:,"flow"] = 0
sdata.loc[:,"width1"] = 5.
sdata.loc[:,"width2"] = 5.
sdata.loc[:,"elevup"] = strtop
sdata.loc[:,"elevdn"] = strtop - slope
sdata.loc[:,"outseg"] = sdata.nseg + 1
sdata.loc[m_tr.nrow-1,"outseg"] = 0

sdata_dict = {0:sdata.to_records(index=False)}
for kper in range(1,m_tr.nper):
    kper_sdata = sdata.to_records(index=False)
    kper_sdata["flow"] *= season_mults[kper-1]
    sdata_dict[kper] = kper_sdata

_ = flopy.modflow.ModflowSfr2(m_tr,nstrm=m_tr.nrow,nss=m_tr.nrow,isfropt=m_org.sfr.isfropt,
                          segment_data=sdata_dict,
                          reach_data=rdata.to_records(index=False),ipakcb=m_org.sfr.ipakcb,
                          istcb2=m_org.sfr.istcb2,reachinput=True)

In [ ]:
m_tr.write_input()

In [ ]:
prep_deps.prep_template(tr_d)

In [ ]:
pyemu.os_utils.run("mfnwt {0}".format(tr_nam),cwd=tr_d)

In [ ]:
lst = flopy.utils.MfListBudget(os.path.join(tr_d,tr_nam+".list"))

In [ ]:
flx,vol = lst.get_dataframes(diff=True,start_datetime=m_tr.start_datetime)
flx.plot(subplots=True,figsize=(20,20))

In [ ]:
flx.head()

In [ ]:
mp_files = [f for f in os.listdir(org_d) if "mp" in f or "location" in f]
[shutil.copy2(os.path.join(org_d,f),os.path.join(tr_d)) for f in mp_files]

In [ ]:
for k in range(m_tr.nlay):
    np.savetxt(os.path.join(tr_d,"prsity_layer_{0}.ref".format(k+1)),np.zeros((m_tr.nrow,m_tr.ncol))+0.1,fmt="%15.6E")

In [ ]:
pyemu.os_utils.run("mp6 freyberg.mpsim",cwd=tr_d)

In [ ]:
cb = flopy.utils.CellBudgetFile(os.path.join(tr_d,tr_nam+".cbc"))

In [ ]:
cb.get_unique_record_names()